In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
print(tf.__version__)

2.0.0


Load IMDB dataframe and metadata

In [5]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Shuffling and writing examples to C:\Users\Carlos\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteI5O4QG\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\Carlos\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteI5O4QG\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\Carlos\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteI5O4QG\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\Carlos\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [33]:
# 25k for train and 25k for testing
# These are iterables containing the 25k of sentences and labels
train_data, test_data = imdb['train'], imdb['test']  

# Define the lists that will contain the sentences and labels
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# s and l are tensors so calling numpy()
# we can extract their values 
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())

# Save as numpy array cause need them this way
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [34]:
print(training_sentences[0])
print(training_labels[0:5])

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
[0, 0, 0, 1, 1]


In [35]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
embedding_dim = 16
# If the sentence is longer than this will be truncated
# Truncated using post
max_length = 120
trunc_type = 'post'
oov_token = '<OOV>'

# Instance of Token with desired vocab size
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
# fit on sentences
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
# create the sequences with the tokens
# This gives me my set of intergers per sentence
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, truncating=trunc_type)

Now comes the different part. The Embedding layer is the key to sentiment analisys with tf. You have words in a sentence and often words that have similar meanings are close to each other. So in a movie review, it might say that the movie was dull and boring, or it might say that it was fun and exciting. So what if you could pick a vector in a higher-dimensional space say 16 dimensions, and words that are found together are given similar vectors. Then over time, words can begin to cluster together. The meaning of the words can come from the labeling of the dataset. So in this case, we say a negative review and the words dull and boring show up a lot in the negative review so that they have similar sentiments, and they are close to each other in the sentence. Thus their vectors will be similar.<br>
As the neural network trains, it can then learn these vectors associating them with the labels to come up with what's called an embedding i.e., the vectors for each word with their associated sentiment. The results of the embedding will be a 2D array with the length of the sentence and the embedding dimension for example 16 as its size. So we need to flatten it out in much the same way as we needed to flatten out our images. We then feed that into a dense neural network to do the classification.<br>
The Embedding dimension will be the number of dimensions for the vector representing the word.

In [36]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(tf.keras.layers.Flatten())
#model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(6, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


Often in natural language processing, a different layer type than a flatten is used, and this is a global average pooling 1D. The reason for this is the size of the output vector being fed into the dance. You can use a Global Average Pooling 1D like this, which averages across the vector to flatten it out. Your model summary should look like this, which is simpler and should be a little faster.<br>

In [37]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs=10

model.fit(padded,
         training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 4s 146us/sample - loss: 0.5006 - accuracy: 0.7360 - val_loss: 0.3891 - val_accuracy: 0.8247
Epoch 2/10
25000/25000 [==============================] - 3s 123us/sample - loss: 0.2441 - accuracy: 0.9052 - val_loss: 0.4090 - val_accuracy: 0.8188
Epoch 3/10
25000/25000 [==============================] - 3s 118us/sample - loss: 0.0984 - accuracy: 0.9746 - val_loss: 0.5120 - val_accuracy: 0.8045
Epoch 4/10
25000/25000 [==============================] - 3s 120us/sample - loss: 0.0250 - accuracy: 0.9966 - val_loss: 0.6216 - val_accuracy: 0.8012
Epoch 5/10
25000/25000 [==============================] - 3s 114us/sample - loss: 0.0057 - accuracy: 0.9997 - val_loss: 0.6766 - val_accuracy: 0.8086
Epoch 6/10
25000/25000 [==============================] - 3s 107us/sample - loss: 0.0021 - accuracy: 0.9999 - val_loss: 0.7415 - val_accuracy: 0.8070
Epoch 7/10
25000/25000 [==========================

Now let's visualize the embedding. Start by getting the results of the embedding layer, which is layer 0, then get the weights. To plot the embedding we need to reverse the word index.

In [45]:
e = model.layers[0]
weights = e.get_weights()[0] 
print(weights.shape) # shape: (vocab_size, embedding_dim)

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

(10000, 16)
? ? ? ? ? ? ? b'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all '
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no exp

Now it's time to write the vectors and their metadata auto files. The TensorFlow Projector reads this file type and uses it to plot the vectors in 3D space so we can visualize them. To the vectors file, we simply write out the value of each of the items in the array of embeddings, i.e, the co-efficient of each dimension on the vector for this word. To the metadata array, we just write out the words. If you're working in Colab, this code will download the two files. To now render the results, go to the TensorFlow Embedding Projector on projector.tensorflow.org, press the ''Load data'' button on the left. You'll see a dialog asking you to load data from your computer. Use vector.TSV for the first one, and meta.TSV for the second. Once they're loaded, you should see something like this. Click this ''sphereize data'' checkbox on the top left, and you'll see the binary clustering of the data. 

In [41]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [44]:
# Use this to download the files if you are in Colab
#try:
#    from google.colab import files
#except ImportError:
#    pass
#else:
#    files.download('vecs.tsv')
#    files.download('meta.tsv')

In [42]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 68, 105, 12, 7, 491, 1216]]
